In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Concatenate , Add, Dot, Activation, Lambda
from tensorflow.keras.models import Model

from tensorflow.image import flip_up_down, flip_left_right, rot90
from tensorflow.linalg import normalize

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import math

import matplotlib.pyplot as plt
import sys 

In [2]:
from tensorflow.python.client import device_lib
#tf.disable_v2_behavior()

#print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(tf.__version__ )

Num GPUs Available:  1
2.9.1


## Sorted Filters

In this experiment, I will try creating a new convolutional layer. Each filter F will be decomposed into Fs and Fa, but only Fs will be learned. Fa will be preinitialised to represent an angle from $0..2\pi$.

Knowing that an antisymetric matrix $F_a$ can be parameterised by 4 values : 

$$F_a = \begin{bmatrix}a & b & c\\
d & 0 & -d \\
-c & -b & -a
\end{bmatrix}$$

I can use Sobel filters (G)  to calulate the dominant angle of $F_a$.

$$G_x = \begin{bmatrix}1& 0 & -1\\
2 & 0 & -2 \\
1 & 0 & -1
\end{bmatrix} * F_a \ \text{   and    } \ G_y = \begin{bmatrix}1& 2 & 1\\
0 & 0 & 0 \\
-1 & -2 & -1
\end{bmatrix} * F_a $$

Which can be writen as:

$$ G_x = 2a - 2c + 4d $$
$$ G_y = 2a+4b+2c$$

And knowing that the Sobel angle $\theta$ when applied to a filter is 


$$ \theta = \tan^{-1} \left(\frac{G_y}{G_x} \right)=  \tan^{-1} \left(\frac{a+2b+c}{a-c+2d} \right)   $$


I can then calculate a value of $d$ in such a way that for any given value of $a, b, c$, the Sobel angle of $F_a$ is $\theta$

By inspection of Sobel filters, I estimate the parameters for a given derivative filter to be

\begin{dcases}
    a &= -\sqrt{2} \cos \left(\theta - \frac{9 \pi}{4} \right) \\
    b &= -2\sin(\theta) \\
    c &=  -\sqrt{2} \sin \left(\theta - \frac{9 \pi}{4} \right) \\
    d &= -2\cos(\theta)\\
\end{dcases}

With this, I can initialise $a, b$ and $c$ sequentialy, this way the filter are sorted by their dominant orientation.

To ensure that I only learn a symetric filter $F_s$, I will make use of the fact that $F_s$ can be parameterised by 3 values : 

$$F_s = \begin{bmatrix}\alpha & \beta & \alpha\\
 \beta & \gamma &  \beta \\
\alpha &  \beta & \alpha
\end{bmatrix}$$

So, by simply learning $\alpha,\beta$ and $\gamma$, I will force the $F_s$ to remain symetric


In [87]:
class SortedConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=(3,3), padding = 'VALID', strides = (1, 1), activation=None, use_bias = True):
        super(SortedConv2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = activation
        self.padding = padding
        self.kernel_initializer = tf.keras.initializers.GlorotNormal(seed=5)
        self.param_initializer =tf.keras.initializers.GlorotNormal(seed=5)

        self.bias_initializer = tf.initializers.Zeros()
        self.strides = strides
        self.use_bias = use_bias

        self.w_a = None     # AntiSymetric kernel weights
        self.w_s = None     # Symetric kernel weights

        self.sym_param_a = None
        self.sym_param_b= None
        self.sym_param_c = None

        self.bias = None

        self.scale = None

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "padding": self.padding,
            "strides": self.strides,
            "activation": self.activation,
            "use_bias": self.use_bias,
        })
        return config

    def get_weights(self):
        return tf.stack([self.w_a, self.w_s])

    def build(self, input_shape):
        *_, n_channels = input_shape

        #####  BUILD Fa   ##### 
        t = tf.repeat([tf.expand_dims(tf.linspace(0.0, 2.0*math.pi, self.filters, axis=0),axis=0)], n_channels, axis=1)

        a = -tf.math.sqrt(2.0)*tf.math.cos(t - 9*math.pi/4)
        b = -2*tf.math.sin(t)
        c = -tf.math.sqrt(2.0)*tf.math.sin(t - 9*math.pi/4)
        d = -2*tf.math.cos(t)
        
        self.scale = tf.Variable(initial_value = tf.keras.initializers.Constant(value=1)(shape=(1)), trainable=True)
        self.w_a   = tf.Variable(tf.stack([tf.concat([a,b,c], axis=0) , 
                                 tf.concat( [d,tf.zeros([1, n_channels, self.filters]), -d], axis=0),
                                 tf.concat( [-c, -b, -a], axis=0)])  , trainable=False)

        #####  BUILD Fs   ##### 
        self.sym_param_a = tf.Variable(
            initial_value=self.param_initializer(shape=(1,
                                                        n_channels,
                                                        self.filters),
                                 dtype='float32'), trainable=True)
        self.sym_param_b = tf.Variable(
            initial_value=self.param_initializer(shape=(1,
                                                        n_channels,
                                                        self.filters),
                                 dtype='float32'), trainable=True)
        self.sym_param_c = tf.Variable(
            initial_value=self.param_initializer(shape=(1,
                                                        n_channels,
                                                        self.filters),
                                 dtype='float32'), trainable=True)

        if self.use_bias:
            self.bias = tf.Variable(
                initial_value=self.bias_initializer(shape=(self.filters,), 
                                                    dtype='float32'),
                trainable=True)


    def call(self, inputs, training=None):

        x_a =  tf.nn.conv2d(inputs, filters=self.w_a * self.scale, strides=self.strides, 
                          padding=self.padding)

        self.w_s  = tf.stack([tf.concat([self.sym_param_a, self.sym_param_b, self.sym_param_a], axis=0), 
                              tf.concat([self.sym_param_b, self.sym_param_c, self.sym_param_b], axis=0),
                              tf.concat([self.sym_param_a, self.sym_param_b, self.sym_param_a], axis=0)])

        x_s =  tf.nn.conv2d(inputs, filters=self.w_s, strides=self.strides, 
                          padding=self.padding)

        if self.use_bias:
            x_s = x_s + self.bias

        return tf.stack([x_a, x_s])
        
layer_a = SortedConv2D(64)


In [44]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from scipy.ndimage import rotate

#https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/

def load_img(img_path,img_shape):
    
    img_rows=img_shape[0]
    img_cols=img_shape[1]
    #num_channel=img_shape[2]

    img = image.load_img(img_path , target_size=(img_rows, img_cols))    
    img = image.img_to_array(img) 

    img = np.expand_dims(img, axis=0)


    return img


In [93]:
img  = load_img('test_images/piano_zoom.png', img_shape=(224,224))


layer_a(img)
print(layer_a.get_weights().shape)
print(layer_a.get_weights()[0,:,:,0,0])

(2, 3, 3, 3, 64)
tf.Tensor(
[[-0.99999994 -0.          1.        ]
 [-2.          0.          2.        ]
 [-1.          0.          0.99999994]], shape=(3, 3), dtype=float32)
